In [1]:
import glob
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
import numpy as np
from itertools import chain 
from IPython.display import clear_output
from ipywidgets import widgets
from IPython.display import display
from sklearn.model_selection import train_test_split
import os
from sklearn.neural_network import MLPClassifier
import numpy as np
import scipy
%autosave 60

Autosaving every 60 seconds


In [2]:
#display(text)
output = None

In [3]:
def progress(count,total):
        global output
        
        #display(widgets.IntSlider())
        #text = widgets.Text()

        with output:
            print "{0:.2f}".format(count*100.0/total)+" %"
            print '|'+'='*int(count*100.0/total)+'>'+'.'*(100-int(count*100.0/total))+'|'
        with output:
            clear_output(wait=True)

In [4]:
# plt.plot(k.values())
# plt.title("Accel readings in t sec time diff")
# plt.show()

##  DATA PREPROCESSING

In [5]:
path = '/home/rakesh/Desktop/zCap/public_dataset/HMOG/'#100669/'
out_path2 = '/home/rakesh/OUTPUT100_21/'

In [6]:
def extract(d):
    
    '''
    'x_min_acc','x_max_acc','x_mean_acc',
           'y_min_acc','y_max_acc','y_mean_acc',
           'z_min_acc','z_max_acc','z_mean_acc',
           'orient_acc_mode'
    '''
    
    x0,x1,x2 = d[0].min(),d[0].max(),d[0].mean()       
    x3,x4,x5 = d[1].min(),d[1].max(),d[1].mean()
    x6,x7,x8 = d[2].min(),d[2].max(),d[2].mean()
    x9 = d[3].mode()[0]
    l = [x0,x1,x2,x3,x4,x5,x6,x7,x8,x9]
    
    return [float(i) for i in l]

In [7]:
def extract_touch(df):
    '''
    INPUT
    0 = pointercount, 1 = actionID, 2 = X , 3 = Y , 4 = pressure , 5 = contact_size, 6 = phone_orient,
    7 = gesture_id , 8 = taskid
    '''
    
    x0,x1 = df[0].mode()[0], df[1].mode()[0]    ## pointer count, action
    x2,x3,x4 = df[2].mean(),df[2].max(),df[2].min()      # x-axis
    x5,x6,x7 = df[3].mean(), df[3].max(), df[3].min()  # y-axis
    x8 = df[4].mean()                ## pressure
    x9 = df[6].mode()[0]                ##phone orientation   
    x10 = df[7].mode()[0]                         # gesture
    x11 = df[8].mode()[0]                 # task
    #x13 = len(df)
    tuple_list = [x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11]
    
    return tuple_list
    

In [8]:
def concat(path,filter_file,idx):
    print filter_file
    df = None
    l = []
    
    for session in glob.glob(path+'*'):
        
        for f in glob.glob(session+'/*'):
        
            if f.split('/')[-1]==filter_file:
                df = pd.read_csv(f,header=None)
                l.append(df)
    df = pd.concat(l)
    df[df.shape[1]] = pd.Series( [idx for _ in range(len(df))], index=df.index)
    return df.sort_values(by=[0]) ## sort by absolute timestamp

In [9]:
def process_acc_gyro(df,gap):
    
    start = df.iloc[0][0]
    db_gyro = []
    db_acc = []
    db_touch =[]
    count2=0
    tmp_touch = []
    tmp_gyro = []
    tmp_acc = []
    count = 0
    lengthDF = len(df)
    
    for index in range(0,lengthDF):

        if index%10000==0:
            progress(index,lengthDF)
            #print "{:.2f} and {}".format((index*1.0)/lengthDF,index),"|",

        row = df.iloc[index].tolist()

        cur_time = row[0]
        #print row 
        #break
        if cur_time<start:
            print "Yes"
        
        if cur_time>=start and cur_time<= start+gap:
            count+=1
            if row[11]==99999:
                tmp_touch.append([row[3]]+row[5:11]+[row[12],row[13]])
                #print tmp_touch
                #break
            if row[7]==0:                      # acc
                tmp_acc.append(row[3:7])

            if row[7]==1:                      # gyro
                tmp_gyro.append(row[3:7])


        else:
            start = cur_time

            count+=1

            if len(tmp_acc)!=0:            ## ACC
                d = pd.DataFrame(tmp_acc)
                db_acc.append(extract(d))
            else:
                db_acc.append([])



            if len(tmp_gyro)!=0:                ## GYRO
                d = pd.DataFrame(tmp_gyro)

                db_gyro.append(extract(d))
            else:
                db_gyro.append([])
                
            if len(tmp_touch)!=0:            ## TOUCH
                d = pd.DataFrame(tmp_touch)
                db_touch.append(extract_touch(d))
            else:
                if len(db_touch)==0:
                    count2+=1
                    db_touch.append([])
                    
                else:
                    db_touch.append(db_touch[-1]) ## donot make empty


            tmp_gyro = []
            tmp_acc = []
            tmp_touch =[]

            if row[7]==0:                      # acc
                tmp_acc.append(row[3:7])

            if row[7]==1:                      # gyro
                tmp_gyro.append(row[3:7])
            
            if row[11]==99999:
                tmp_touch.append([row[3]]+row[5:11]+[row[12],row[13]])
            #if tmp.append(row[1:])

    print count 
    print "Touch List empty : "+str(count2)
    return db_acc,db_gyro,db_touch

In [10]:
def construct(path,out_path,gap):
    global output
    for user in glob.glob(path+'*'):
        user_id = user.split('/')[-1]
        
        if os.path.exists(out_path+str(user_id)+'_acc_gyro.csv'):
            print str(user_id)+'_acc_gyro.csv',"skipped"
            continue
        
        print "User",user_id
        
        acc = concat(user+'/','Accelerometer.csv',idx = 0)  
        gyro = concat(user+'/','Gyroscope.csv',idx=1)
        touch = concat(user+'/','TouchEvent.csv',idx=99999)
        activity = concat(user+'/','Activity.csv',idx=88888).set_index(0)# ,inplace = True)
        
        # add activity with touch
        gesture_7 = []
        task_8 = []
        '''
        1, 7, 13, 19: Reading + Sitting
        2, 8, 14, 20: Reading + Walking
        3, 9, 15, 21: Writing + Sitting!
        4, 10, 16, 22: Writing + Walking
        5, 11, 17, 23: Map + Sitting
        '''
        task_map = {1:1,7:1,13:1,
                    2:2,8:2,14:2,20:2,
                    3:3,9:3,15:3,21:3,
                    4:4,10:4,16:4,22:4,
                    5:5,11:5,17:5,23:5,
                    6:6, 12:6, 18:6, 24:6}
        count =0
        for id_ in range(len(touch)):
            act_id = touch.iloc[id_][2]
            try:
                gesture_7.append(activity.loc[act_id][7])
                task_8.append(task_map[activity.loc[act_id][8]])
            except:
                count+=1
                gesture_7.append(np.NaN)
                task_8.append(np.NaN)
                
        touch[12] = pd.Series( [i for i in gesture_7])
        touch[13] = pd.Series( [i for i in task_8])
        print "miss ; "+str(count)
        touch = touch.dropna()
        
        df = pd.concat([acc, gyro,touch], axis=0).sort_values(by=[0]).reset_index(drop=True)
        
        output = widgets.Output()
        display(output)
        db_acc,db_gyro,db_touch = None,None,None
        try:
            db_acc,db_gyro,db_touch = process_acc_gyro(df,gap)
        except:
            continue
        
        X1 = pd.DataFrame(db_acc)
        X2 = pd.DataFrame(db_gyro)
        X3 = pd.DataFrame(db_touch)
        
        print X1.shape
        print X2.shape
        print X3.shape
        acc_gyro = pd.concat([X1, X2,X3], axis=1)
        
        print acc_gyro.shape
        print "Saving...",user_id
        #return acc_gyro
        acc_gyro.to_csv(out_path+str(user_id)+'_all.csv',sep = ',',header=False)

construct(path,out_path2,gap=100)
#construct(path,'/home/rakesh/OUTPUT/',gap=10)
#os.system('shutdown -s')

User 803262
Accelerometer.csv
Gyroscope.csv
TouchEvent.csv
Activity.csv
miss ; 2644


Output()

3208227
Touch List empty : 1
(144837, 10)
(144837, 10)
(144837, 12)
(144837, 32)
Saving... 803262
User 990622
Accelerometer.csv
Gyroscope.csv
TouchEvent.csv
Activity.csv
miss ; 1050


Output()

2353207
Touch List empty : 1
(104480, 10)
(104480, 10)
(104480, 12)
(104480, 32)
Saving... 990622
User 472761
Accelerometer.csv
Gyroscope.csv
TouchEvent.csv
Activity.csv
miss ; 521


...

Output()

2620153
Touch List empty : 1
(120510, 10)
(120510, 10)
(120510, 12)
(120510, 32)
Saving... 865881
User 218719
Accelerometer.csv
Gyroscope.csv
TouchEvent.csv
Activity.csv
miss ; 999


Output()

User 785899
Accelerometer.csv
Gyroscope.csv
TouchEvent.csv
Activity.csv


KeyboardInterrupt: 

In [ ]:
t

In [ ]:
t

In [ ]:
#acc_gyro.to_csv('100669_acc_gyro.csv',sep = ',',header=False)

In [ ]:
# df = df.interpolate()
# df.shape

In [ ]:
#df.to_csv('100669.csv',sep = ',',header=False)

##   TOuchEvent

In [ ]:
path = '/home/rakesh/Desktop/zCap/public_dataset/HMOG/100669/'

filter_file = 'TouchEvent.csv'

In [ ]:
def concat(filter_file):
    df = None
    l = []
    
    for session in glob.glob(path+'*'):
    
        for f in glob.glob(session+'/*'):
        
            if f.split('/')[-1]==filter_file:
                df = pd.read_csv(f,header=None)
                l.append(df)
    return pd.concat(l).sort_values(by=[0]) ## sort by absolute timestamp

In [ ]:
mdf = concat(filter_file)

In [ ]:
sum(Counter(mdf[2]).values())/113

In [ ]:
#l1 = acc[2].tolist()+gyro[2].tolist()
#sum(Counter(l1).values())/len(Counter(l1))

In [ ]:
mdf = mdf.drop(columns=[1,2])

In [ ]:
start = mdf.iloc[0][0]

gap = 10       # gap of 10ms
count = 0
touch_list = []
tmp = []
length = len(mdf)

output = widgets.Output()
display(output)

for index in range(0,length):
    
    if index%2000==0:
        progress(index,length)
        #print "{:.2f} and {}".format((index*1.0)/length,index),"|",
    
    
    row = mdf.iloc[index].tolist()
    count+=1
    cur_time = row[0]
    if cur_time<start:
        print "Yes"
    
    if cur_time>=start and cur_time<= start+gap:
        
        tmp.append(row[1:])

    else:
        
        start = cur_time
        count+=1
        if len(tmp)!=0:
            
            df = pd.DataFrame(tmp)
            
            x0,x1 = df[0].mode()[0], df[1].mode()[0]    ## pointer count, pointerid
            x2,x3,x4 = df[3].mean(),df[3].max(),df[3].min()      # x-axis
            x5,x6,x7 = df[4].mean(), df[4].max(), df[4].min()  # y-axis
            x8,x9 = df[5].mean(),df[5].max()                ## pressure
            x10,x11 = df[6].mean(),df[6].max()                ##contact size   
            x12 = df[7].mode()[0]                         # phone orientation
            #x13 = len(df)
            tuple_list = [x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12]
            try:
                tuple_list = [float(i) for i in tuple_list]
            except:
                print "\n\n",tuple_list
                break
            #print tuple_list
            touch_list.append(tuple_list)
        tmp = []
        
        tmp.append(row[1:])
print count

In [ ]:
touch = pd.DataFrame(touch_list)
print touch.shape
touch = touch.drop_duplicates()

In [ ]:
touch

## ===============  Merge Files =========================

In [ ]:
input_path = '/home/rakesh/OUTPUT50/'
files = 'all.csv'
features = [
            'x_min_acc','x_max_acc','x_mean_acc',    # 0,1,2
           'y_min_acc','y_max_acc','y_mean_acc',     # 3,4,5
           'z_min_acc','z_max_acc','z_mean_acc',     # 6,7,8
           'orient_acc_mode',                        # 9
           'x_min_gyro','x_max_gryo','x_mean_gryo',  # 10,11,12
           'y_min_gryo','y_max_gryo','y_mean_gryo',  # 13,14,15
           'z_min_gryo','z_max_gryo','z_mean_gryo',  # 16,17,18
           'orient_gryo_mode',                       # 19
            'pointercount mode','actionId mode',     # 20,21
            'touch_x_mean','touch_x_max','touch_x_min', #22,23,24
            'touch_y_mean','touch_y_max','touch_y_min', #25,26,27
            'pressure_mean','phone_orien mode','gesture mode','task mode' # 28,29,30,31
            ]
drop = [10,11,12,16,17,18,28,20,13,14,15]

In [ ]:
df1 = pd.read_csv(input_path+'803262_'+files ,header = None).drop(columns=[0]) ## drop index column
df2 = pd.read_csv(input_path+'472761_'+files ,header = None).drop(columns=[0])
df3 = pd.read_csv(input_path+'489146_'+files ,header = None).drop(columns=[0])
df4 = pd.read_csv(input_path+'990622_'+files ,header = None).drop(columns=[0])
df5 = pd.read_csv(input_path+'621276_'+files ,header = None).drop(columns=[0])
df6 = pd.read_csv(input_path+'856401_'+files ,header = None).drop(columns=[0])

In [ ]:
df1 = df1.drop(columns = drop)
df2 = df2.drop(columns = drop)
df3 = df3.drop(columns = drop)
df4 = df4.drop(columns = drop)
df5 = df5.drop(columns = drop)
df6 = df6.drop(columns = drop)

print "df1",df1.shape
print "df2",df2.shape
print "df3",df3.shape
print "df4",df4.shape
print "df5",df5.shape
print "df6",df6.shape

In [ ]:
print "Gunuine dist"
p = df6.shape[0]/5
p = p-(p%1000)
p

In [ ]:
df6 = df6.dropna()
df1 = df1.dropna().iloc[0:p]
df2 = df2.dropna().iloc[0:p]
df3 = df3.dropna().iloc[0:p]
df4 = df4.dropna().iloc[0:p]
df5 = df5.dropna().iloc[0:p]

In [ ]:
user = '803262'
Imposters = ['472761','489146','990622','621276','856401']
print "User : ",user
print "Imposters : ", Imposters

In [ ]:
y1 = [0 for i in range(len(df1))]
y2 = [0 for i in range(len(df2))]
y3 = [0 for i in range(len(df3))]
y4 = [0 for i in range(len(df4))]
y5 = [0 for i in range(len(df5))]
y6 = [1 for i in range(len(df6))]

y = y1+y2+y3+y4+y5+y6

In [ ]:
X = pd.concat([df1,df2,df3,df4,df5,df6], axis=0)
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,random_state=42, test_size=0.15)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier( alpha=1e-4,
                    hidden_layer_sizes=(15,35,10), random_state=1,verbose = True,max_iter=600)

clf.fit(X_train, y_train)                         

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
(y_pred==np.array(y_test)).mean()

In [ ]:
scipy.io.savemat('856401.mat', mdict={'X_train': X_train.values,
                                   'X_test':X_test.values,
                                   'Y_train':y_train,
                                    'Y_test': y_test})

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(y_test, y_pred, pos_label=1)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
EER

In [ ]:
fnr[np.nanargmin(np.absolute((fnr - fpr)))]

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler,MinMaxScaler
# from sklearn.manifold import TSNE
# from matplotlib import pyplot as plt

# #x = MinMaxScaler(feature_range=(0, 1)).fit_transform(X_train[0:5000])
# g = TSNE(n_components=2).fit_transform(X_train[0:5000])
# X_pca = g#pca.transform(x)
# target_ids = range(4)
# y = y_train[0:5000]

# plt.figure(figsize=(8, 8))
# # for i, c, label in zip(target_ids, 'rgbc', target_ids):
# #     plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1],
# #                c=c, label=label)
# plt.scatter(X_pca[0:,0],X_pca[0:,1],y)
# plt.legend()
# plt.show()
# X_pca

In [ ]:
## ====================   classifier =============

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1,1)
    def forward(self, x):
        x = self.fc1(x)
        return x

In [ ]:
        task_map = {1:1,7:1,13:1,
                    2:2,8:2,14:2,20:2,
                    3:3,9:3,15:3,21:3,
                    4:4,10:4,16:4,22:4,
                    5:5,11:5,17:5,23:5}

In [ ]:
task_map[20]

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2, whiten=True)
pca.fit(X)

In [ ]:
X_pca = pca.transform(X)

In [ ]:
target_ids = range(4)
target_ids

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(6, 5))
for i, c, label in zip(target_ids, 'rgbcmykw', iris.target_names):
    plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1],
               c=c, label=label)
plt.legend()
plt.show()

In [ ]:
iris.target_names